## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: HEK293

In [10]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
# create a database to store optuna studies with sqlite backend
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [12]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [13]:
cell_line = CELL_LINES[3]
cell_line

'HEK293'

---

In [14]:
from BIOINF_tesi.data_pipe import Load_Create_Task
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline

In [15]:
from BIOINF_tesi.models import FFNN, CNN, CNN_LSTM
from BIOINF_tesi.models.utils import fit, Param_Search, Kfold_CV

In [49]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

In [50]:
for i in results_dict.keys():
    display(results_dict[i].keys())

dict_keys(['active_E_vs_inactive_E', 'active_P_vs_inactive_P', 'active_E_vs_active_P', 'inactive_E_vs_inactive_P', 'active_EP_vs_inactive_rest'])

dict_keys(['active_E_vs_inactive_E', 'active_P_vs_inactive_P', 'active_E_vs_active_P', 'inactive_E_vs_inactive_P', 'active_EP_vs_inactive_rest'])

dict_keys(['active_E_vs_inactive_E', 'active_P_vs_inactive_P', 'active_E_vs_active_P', 'inactive_E_vs_inactive_P', 'active_EP_vs_inactive_rest'])

dict_keys(['active_E_vs_inactive_E', 'active_P_vs_inactive_P', 'active_E_vs_active_P', 'inactive_E_vs_inactive_P', 'active_EP_vs_inactive_rest'])

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [17]:
task = TASKS[0]
task

'active_E_vs_inactive_E'

In [18]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


### 1. FFNN

In [ ]:
model=FFNN

In [ ]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                augmentation=False,
                model = model,
                device = device,
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP.pt',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST.pt')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [22]:
model=CNN

In [23]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                task=task,
                sequence=True,
                augmentation=False,
                model = model,
                device = device,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:364: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-01 22:21:30,676] Using an existing study with name 'HEK293_active_E_vs_inactive_E_CNN_1' instead of creating a new one.
/opt/conda/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
/opt/conda/lib/python3.7/site-packages/timm/optim/nadam.py:69: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/torch/csrc/utils/python_arg_parser.cpp:1025.)
  grad = grad.add(group['weight_decay'], p.data)


EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-01 23:40:21,918] Trial 2 finished with value: 0.13107826152595628 and parameters: {'n_layers': 2, 'out_channels_l0': 16, 'kernel_size_l0': 5, 'dropout_l0': 0.4, 'out_channels_l1': 32, 'kernel_size_l1': 15, 'dropout_l1': 0, 'optimizer': 'Nadam', 'lr': 1.4657684411959107e-05, 'weight_decay': 0.000291528602349809}. Best is trial 2 with value: 0.13107826152595628.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.13107826152595628
  Params: 
    dropout_l0: 0.4
    dropout_l1: 0
    kernel_size_l0: 5
    kernel_size_l1: 15
    lr: 1.4657684411959107e-05
    n_layers: 2
    optimizer: Nadam
    out_channels_l0: 16
    out_channels_l1: 32
    weight_decay: 0.000291528602349809

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.13105092640936397



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.12347


In [24]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [25]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 3. CNN-LSTM

In [ ]:
model=CNN_LSTM

In [ ]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                task=task,
                sequence=True,
                augmentation=False,
                model = model,
                device = device,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP.pt',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST.pt')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [26]:
task = TASKS[1]
task

'active_P_vs_inactive_P'

In [27]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RobustScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


---

### 1. FFNN

In [ ]:
model=FFNN

In [ ]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                augmentation=False,
                model = model,
                device = device,
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [28]:
model=CNN

In [29]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                task=task,
                sequence=True,
                augmentation=False,
                model = model,
                device = device,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:364: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-01 23:46:16,062] A new study created in RDB with name: HEK293_active_P_vs_inactive_P_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-02 00:24:01,025] Trial 0 finished with value: 0.1348483177054606 and parameters: {'n_layers': 4, 'out_channels_l0': 32, 'kernel_size_l0': 5, 'dropout_l0': 0.3, 'out_channels_l1': 32, 'kernel_size_l1': 15, 'dropout_l1': 0.4, 'out_channels_l2': 96, 'kernel_size_l2': 11, 'dropout_l2': 0, 'out_channels_l3': 512, 'kernel_size_l3': 15, 'dropout_l3': 0.4, 'optimizer': 'Nadam', 'lr': 0.014453172898488085, 'weight_decay': 0.08938146525880786}. Best is trial 0 with value: 0.1348483177054606.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-02 01:53:11,794] Trial 2 finished with value: 0.13615643472786332 and parameters: {'n_layers': 1, 'out_channels_l0': 64, 'kernel_size_l0': 15, 'dropout_l0': 0.4, 'optimizer': 'Nadam', 'lr': 0.000919321652340543, 'weight_decay': 0.018459367261671456}. Best is trial 0 with value: 0.15039599727236305.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.15039599727236305
  Params: 
    dropout_l0: 0.3
    dropout_l1: 0
    dropout_l2: 0
    dropout_l3: 0.5
    kernel_size_l0: 15
    kernel_size_l1: 11
    kernel_size_l2: 5
    kernel_size_l3: 11
    lr: 0.00043511212493037684
    n_layers: 4
    optimizer: Nadam
    out_channels_l0: 64
    out_channels_l1: 96
    out_channels_l2: 256
    out_channels_l3: 256
    weight_decay: 0.009350404044320703

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.1850664837674507



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.167


In [30]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [31]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 3. CNN-LSTM

In [ ]:
model=CNN_LSTM

In [ ]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                task=task,
                sequence=True,
                augmentation=False,
                model = model,
                device = device,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP.pt',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST.pt')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [32]:
task = TASKS[2]
task

'active_E_vs_active_P'

In [33]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RobustScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


---
### 1. FFNN

In [ ]:
model=FFNN

In [ ]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                augmentation=False,
                random_state=32,
                model = model,
                device = device,
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP.pt',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST.pt')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [34]:
model=CNN

In [35]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                task=task,
                sequence=True,
                augmentation=False,
                model = model,
                device = device,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP.',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:364: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-02 02:09:00,384] A new study created in RDB with name: HEK293_active_E_vs_active_P_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-02 02:10:55,751] Trial 0 finished with value: 0.3416048237476809 and parameters: {'n_layers': 4, 'out_channels_l0': 32, 'kernel_size_l0': 11, 'dropout_l0': 0.3, 'out_channels_l1': 32, 'kernel_size_l1': 5, 'dropout_l1': 0, 'out_channels_l2': 64, 'kernel_size_l2': 11, 'dropout_l2': 0.5, 'out_channels_l3': 256, 'kernel_size_l3': 11, 'dropout_l3': 0.4, 'optimizer': 'RMSprop', 'lr': 0.0005172348566156131, 'weight_decay': 0.0004725984529469053}. Best is trial 0 with value: 0.3416048237476809.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-02 02:38:40,462] Trial 2 finished with value: 0.6657845210609502 and parameters: {'n_layers': 1, 'out_channels_l0': 16, 'kernel_size_l0': 15, 'dropout_l0': 0.3, 'optimizer': 'Nadam', 'lr': 1.7337110473716872e-05, 'weight_decay': 0.00017521267344899094}. Best is trial 1 with value: 0.6806505749636211.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.6806505749636211
  Params: 
    dropout_l0: 0.2
    kernel_size_l0: 11
    lr: 0.01102311917927527
    n_layers: 1
    optimizer: Nadam
    out_channels_l0: 32
    weight_decay: 0.00406789022283829

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.697818353130553



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.70037


In [36]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [37]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [38]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [39]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RobustScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


---
### 1. FFNN

In [ ]:
model=FFNN

In [ ]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                augmentation=False,
                model = model,
                device = device,
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP.pt',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST.pt')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [40]:
model=CNN

In [41]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                task=task,
                sequence=True,
                augmentation=False,
                model = model,
                device = device,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:364: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-02 02:40:43,041] A new study created in RDB with name: HEK293_inactive_E_vs_inactive_P_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-02 02:55:04,762] Trial 0 finished with value: 0.5875682201309683 and parameters: {'n_layers': 2, 'out_channels_l0': 64, 'kernel_size_l0': 11, 'dropout_l0': 0, 'out_channels_l1': 64, 'kernel_size_l1': 15, 'dropout_l1': 0.5, 'optimizer': 'RMSprop', 'lr': 0.0014793191250024433, 'weight_decay': 0.0003765887943208693}. Best is trial 0 with value: 0.5875682201309683.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-02 05:12:07,287] Trial 2 finished with value: 0.39998992787157167 and parameters: {'n_layers': 3, 'out_channels_l0': 32, 'kernel_size_l0': 11, 'dropout_l0': 0.2, 'out_channels_l1': 64, 'kernel_size_l1': 11, 'dropout_l1': 0.4, 'out_channels_l2': 256, 'kernel_size_l2': 15, 'dropout_l2': 0, 'optimizer': 'Nadam', 'lr': 3.587678379348472e-05, 'weight_decay': 0.00010737479902211135}. Best is trial 0 with value: 0.551413081128876.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.551413081128876
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.4
    kernel_size_l0: 11
    kernel_size_l1: 5
    lr: 1.0767901988909999e-05
    n_layers: 2
    optimizer: Adam
    out_channels_l0: 16
    out_channels_l1: 64
    weight_decay: 0.00012669650397020672

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.554318851393717



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.57613


In [42]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [43]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [44]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [45]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RobustScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


---
### 1. FFNN

In [ ]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                augmentation=False,
                model = model,
                device = device,
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP.pt',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST.pt')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [ ]:
model=CNN

In [46]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                task=task,
                sequence=True,
                augmentation=False,
                model = model,
                device = device,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:364: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-02 05:25:28,695] A new study created in RDB with name: HEK293_active_EP_vs_inactive_rest_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-02 05:35:03,663] Trial 0 finished with value: 0.11765738498789352 and parameters: {'n_layers': 2, 'out_channels_l0': 16, 'kernel_size_l0': 5, 'dropout_l0': 0.4, 'out_channels_l1': 64, 'kernel_size_l1': 11, 'dropout_l1': 0.4, 'optimizer': 'Adam', 'lr': 0.05889370551019785, 'weight_decay': 0.004895775338496469}. Best is trial 0 with value: 0.11765738498789352.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-02 08:17:00,614] Trial 2 finished with value: 0.12047619047619053 and parameters: {'n_layers': 4, 'out_channels_l0': 16, 'kernel_size_l0': 11, 'dropout_l0': 0.3, 'out_channels_l1': 96, 'kernel_size_l1': 11, 'dropout_l1': 0.5, 'out_channels_l2': 96, 'kernel_size_l2': 11, 'dropout_l2': 0.4, 'out_channels_l3': 256, 'kernel_size_l3': 15, 'dropout_l3': 0, 'optimizer': 'Adam', 'lr': 0.0019583020710715222, 'weight_decay': 0.04489594159264728}. Best is trial 1 with value: 0.15470304237009863.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.15470304237009863
  Params: 
    dropout_l0: 0.3
    dropout_l1: 0
    kernel_size_l0: 15
    kernel_size_l1: 5
    lr: 7.887983736378855e-05
    n_layers: 2
    optimizer: RMSprop
    out_channels_l0: 32
    out_channels_l1: 96
    weight_decay: 0.026545012591882536

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.15223539197959984



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.15168


In [47]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [48]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)